# Querying and Exporting for Analysis

[David J. Thomas](mailto:dave.a.base@gmail.com), [thePortus.com](http://thePortus.com)<br />
Instructor of Ancient History and Digital Humanities,<br />
Department of History,<br />
[University of South Florida](https://github.com/usf-portal)

---

## This workbook will...

* Query the database for information in different ways
* Export the resulting information for use in several different manners

---

## 1) Import Module Dependencies

The cell below loads all other Python packages needed. You **must** run this before any other cells.

In [ ]:
import os
import sqlalchemy as sql
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
from IPython.display import clear_output
import dhelp

## 2) (Re)declare Database Models

In [ ]:
engine = sql.create_engine('sqlite:///charters.db', echo=False)
Base = declarative_base()
    

class Charter(Base):
    __tablename__ = 'charters'

    id = sql.Column(sql.String, primary_key=True)
    description = sql.Column(sql.String)
    sawyer = sql.Column(sql.Integer)
    birch = sql.Column(sql.Integer)
    kemble = sql.Column(sql.Integer)
    british_academy = sql.Column(sql.String)
    source_used = sql.Column(sql.String)
    archive = sql.Column(sql.String)
    language = sql.Column(sql.String)
    date = sql.Column(sql.Integer)
    scholarly_date = sql.Column(sql.String)
    scholarly_date_low = sql.Column(sql.Integer)
    scholarly_date_high = sql.Column(sql.Integer)
    scholarly_date_avg = sql.Column(sql.Float)
    text = sql.Column(sql.Text)
    notes = sql.Column(sql.Text)
    asc_source = sql.Column(sql.String)
    pase_source = sql.Column(sql.String)
    pase_witnesses = sql.Column(sql.String)
    
    witnesses = sql.orm.relationship('Person', secondary='charter_witnesses', back_populates='charters')
    
    @property
    def record(self):
        """Gets entry data in dictionary format."""
        return {
            'id': self.id,
            'description': self.description,
            'sawyer': self.sawyer,
            'birch': self.birch,
            'kemble': self.kemble,
            'british_academy': self.british_academy,
            'source_used': self.source_used,
            'archive': self.archive,
            'language': self.language,
            'date': self.date,
            'scholarly_date': self.scholarly_date,
            'scholarly_date_low': self.scholarly_date_low,
            'scholarly_date_high': self.scholarly_date_high,
            'scholarly_date_avg': self.scholarly_date_avg,
            'text': self.text,
            'notes': self.notes,
            'asc_source': self.asc_source,
            'pase_source': self.pase_source,
            'pase_witnesses': self.pase_witnesses
        }
    
    @property
    def coappearances(self):
        """Gives a list of person -> person relationships formed through coappearance on the charter."""
        witnesses = []
        for witness_index, witness in enumerate(self.witnesses):
            if witness_index < len(self.witnesses):
                for cowitness_index, cowitness in enumerate(self.witnesses[witness_index + 1:]):
                    witnesses.append({'source': witness, 'target': cowitness})
        return witnesses
                    
    
class Person(Base):
    __tablename__ = 'people'
    
    id = sql.Column(sql.String, primary_key=True)
    description = sql.Column(sql.String)
    link = sql.Column(sql.String)
    
    charters = sql.orm.relationship('Charter', secondary='charter_witnesses', back_populates='witnesses')
    
    @property
    def record(self):
        """Gets entry data in dictionary format."""
        return {
            'id': self.id,
            'description': self.description,
            'link': self.link
        }
    
    @property
    def earliest_appearance(self):
        """Returns the date of the earliest charter features said person."""
        earliest_charter = None
        for charter in self.charters:
            if not earliest_charter:
                earliest_charter = charter.scholarly_date_avg
            else:
                if charter.scholarly_date_avg < earliest_charter:
                    earliest_charter = charter.scholarly_date_avg
        return earliest_charter

    
class CharterWitness(Base):
    __tablename__ = 'charter_witnesses'
    charter_id = sql.Column(sql.String, sql.ForeignKey('charters.id'), primary_key=True) 
    person_id = sql.Column(sql.String, sql.ForeignKey('people.id'), primary_key=True)
    role = sql.Column(sql.String)
    link = sql.Column(sql.String)
    
    @property
    def record(self):
        """Gets entry data in dictionary format."""
        return {
            'charter_id': self.charter_id,
            'person_id': self.person_id,
            'role': self.role,
            'link': self.link
        }


print('Database Configured Successfully')

## Gephi Network Node/Edge CSVs

In [ ]:
# first, we need to open a session with the local database
Session = sessionmaker(bind=engine)
session = Session()

fieldnames = ['id', 'label', 'description', 'class', 'archive', 'language', 'start_date', 'end_date', 'text', 'notes', 'link']
data_dicts = []

# add charter data in gephi-friendly node format
for charter in session.query(Charter):
    data_dicts.append({
        'id': charter.id,
        'label': charter.id,
        'description': charter.description,
        'class': 'Charter',
        'archive': charter.archive,
        'language': charter.language,
        'start_date': charter.scholarly_date_avg,
        'end_date': 1000,
        'text': charter.text,
        'notes': charter.notes,
        'link': charter.asc_source
    })

# add person data in gephi-friendly node format
for person in session.query(Person):
        data_dicts.append({
        'id': person.id,
        'label': person.id + person.description,
        'description': person.description,
        'class': 'Person',
        'archive': None,
        'language': None,
        'start_date': person.earliest_appearance,
        'end_date': 1000,
        'text': None,
        'notes': None,
        'link': person.link
    })

# save data to file and reset fieldnames and data_dicts
dhelp.CSVFile('export/charter-witnesses-nodes.csv', options={'overwrite': True}).save(data_dicts, fieldnames=fieldnames)      
fieldnames = ['source', 'target', 'type', 'weight', 'role', 'start_date', 'end_date']
data_dicts = []

# add person -> charter edge data in gephi-friendly format
for charter_witness in session.query(CharterWitness):
    charter = session.query(Charter).filter(Charter.id == charter_witness.charter_id)[0]
    data_dicts.append({
        'source': charter_witness.person_id,
        'target': charter_witness.charter_id,
        'type': 'Undirected',
        'weight': 1.0,
        'role': charter_witness.role,
        'start_date': charter.scholarly_date_avg,
        'end_date': 1000
    })

# save data to file and close session
dhelp.CSVFile('export/charter-witnesses-edges.csv', options={'overwrite': True}).save(data_dicts, fieldnames=fieldnames)
session.close()

print('Done exporting data for Gephi')